In [10]:
import numpy as np

from keras.models import Model
from keras.layers import Input, Dense, Flatten, Embedding
from keras.layers.merge import concatenate
from keras.callbacks import TensorBoard
from keras.utils.np_utils import to_categorical

dictionary_size = 100
embeddings_size = 20
num_inputs = 3

inputs = []
embeddings = []

# Prepare several inputs for CBOW.
for i in range(num_inputs):
    inp = Input(shape=(1,), dtype='int32', name='input_{}'.format(i))
    layer = Embedding(input_dim=dictionary_size, output_dim=embeddings_size,
                      input_length=1, name='embeddings_{}'.format(i))(inp)
    # 3D tensor to 2D for further merge.
    layer = Flatten()(layer)
    inputs.append(inp)
    embeddings.append(layer)

# Ready to merge all the embedding layers together.
layer = concatenate(embeddings)

# Put some layers on top. Names here are option as we don't care
# of this layers, so they'll be default ones in case if we omit this parameter.
layer = Dense(embeddings_size * num_inputs, activation='relu',
              name='hidden_1')(layer)
layer = Dense(embeddings_size * num_inputs, activation='relu',
              name='hidden_2')(layer)
layer = Dense(embeddings_size * num_inputs, activation='relu',
              name='hidden_3')(layer)

# Output layer has to have the same dimensions as input embeddings for CBOW.
# Also get a probabilistic output with softmax activation function.
layer = Dense(dictionary_size, activation='softmax', name='output')(layer)

model = Model(inputs=inputs, outputs=layer)

# Whatever you want here.
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

num_samples = 1000

X = [np.random.randint(dictionary_size, size=num_samples) for _ in
     range(num_inputs)]
Y = to_categorical(np.random.randint(dictionary_size, size=num_samples),
                   dictionary_size)

embeddings_layer_names = None # or some list of layers ['embeddings_1']

tb_callback = TensorBoard(histogram_freq=10, write_graph=True,write_images=True)
                        

# Ready to fit.
model.fit(X, Y, epochs=400, batch_size=512, callbacks=[tb_callback])

INFO:tensorflow:Summary name embeddings_0_2/embeddings:0 is illegal; using embeddings_0_2/embeddings_0 instead.
INFO:tensorflow:Summary name embeddings_0_2/embeddings:0 is illegal; using embeddings_0_2/embeddings_0 instead.
INFO:tensorflow:Summary name embeddings_1_2/embeddings:0 is illegal; using embeddings_1_2/embeddings_0 instead.
INFO:tensorflow:Summary name embeddings_1_2/embeddings:0 is illegal; using embeddings_1_2/embeddings_0 instead.
INFO:tensorflow:Summary name embeddings_2_2/embeddings:0 is illegal; using embeddings_2_2/embeddings_0 instead.
INFO:tensorflow:Summary name embeddings_2_2/embeddings:0 is illegal; using embeddings_2_2/embeddings_0 instead.
INFO:tensorflow:Summary name hidden_1_2/kernel:0 is illegal; using hidden_1_2/kernel_0 instead.
INFO:tensorflow:Summary name hidden_1_2/kernel:0 is illegal; using hidden_1_2/kernel_0 instead.
INFO:tensorflow:Summary name hidden_1_2/bias:0 is illegal; using hidden_1_2/bias_0 instead.
INFO:tensorflow:Summary name hidden_1_2/bias

In [5]:
help(TensorBoard)

Help on class TensorBoard in module keras.callbacks:

class TensorBoard(Callback)
 |  Tensorboard basic visualizations.
 |  
 |  This callback writes a log for TensorBoard, which allows
 |  you to visualize dynamic graphs of your training and test
 |  metrics, as well as activation histograms for the different
 |  layers in your model.
 |  
 |  TensorBoard is a visualization tool provided with TensorFlow.
 |  
 |  If you have installed TensorFlow with pip, you should be able
 |  to launch TensorBoard from the command line:
 |  ```
 |  tensorboard --logdir=/full_path_to_your_logs
 |  ```
 |  You can find more information about TensorBoard
 |  [here](https://www.tensorflow.org/versions/master/how_tos/summaries_and_tensorboard/index.html).
 |  
 |  # Arguments
 |      log_dir: the path of the directory where to save the log
 |          files to be parsed by Tensorboard.
 |      histogram_freq: frequency (in epochs) at which to compute activation
 |          histograms for the layers of th

In [11]:
import glob
glob.glob("./logs/*")

['./logs/events.out.tfevents.1490089168.cuda-dx.ds.corp.kelkoo.net',
 './logs/events.out.tfevents.1490085072.cuda-dx.ds.corp.kelkoo.net']